# Querying the Notes & Queries Database

The *Notes & Queries* contains search text for 19th century issues of *Notes & Queries* as well as extracted index information which provides a slightly more structured view over the data.

In [1]:
from sqlite_utils import Database

db_name_full = "full_nq.db"
db_full = Database(db_name_full)

What tables are available?

In [2]:
db_full.tables

[<Table metadata (id, date, datetime, series, vol, iss, title, next_id, prev_id, is_index, restricted)>,
 <Table issues (id, content)>,
 <Table issues_fts (id, content)>,
 <Table issues_fts_data (id, block)>,
 <Table issues_fts_idx (segid, term, pgno)>,
 <Table issues_fts_docsize (id, sz)>,
 <Table issues_fts_config (k, v)>,
 <Table pages_metadata (id, page_idx, page_char_start, page_char_end, page_leaf_num, page_num, page_num_conf, page_text)>,
 <Table pages_metadata_fts (id, page_idx, page_text)>,
 <Table pages_metadata_fts_data (id, block)>,
 <Table pages_metadata_fts_idx (segid, term, pgno)>,
 <Table pages_metadata_fts_docsize (id, sz)>,
 <Table pages_metadata_fts_config (k, v)>,
 <Table index_entries (source_id, year, vol, index_term, typ, subsidiary, page_num)>,
 <Table index_entries_fts (source_id, index_term, subsidiary, year, vol, page_num)>,
 <Table index_entries_fts_data (id, block)>,
 <Table index_entries_fts_idx (segid, term, pgno)>,
 <Table index_entries_fts_docsize (id, 

In [54]:
print(db_full.schema)

CREATE TABLE [metadata] (
   [id] TEXT PRIMARY KEY,
   [date] TEXT,
   [datetime] TEXT,
   [series] TEXT,
   [vol] TEXT,
   [iss] TEXT,
   [title] TEXT,
   [next_id] TEXT,
   [prev_id] TEXT,
   [is_index] INTEGER,
   [restricted] TEXT
, [year] INTEGER);
CREATE TABLE [issues] (
   [id] TEXT PRIMARY KEY REFERENCES [metadata]([id]),
   [content] TEXT
);
CREATE VIRTUAL TABLE [issues_fts] USING FTS5 (
    [id], [content],
    tokenize='porter',
    content=[issues]
);
CREATE TABLE 'issues_fts_data'(id INTEGER PRIMARY KEY, block BLOB);
CREATE TABLE 'issues_fts_idx'(segid, term, pgno, PRIMARY KEY(segid, term)) WITHOUT ROWID;
CREATE TABLE 'issues_fts_docsize'(id INTEGER PRIMARY KEY, sz BLOB);
CREATE TABLE 'issues_fts_config'(k PRIMARY KEY, v) WITHOUT ROWID;
CREATE TRIGGER [issues_ai] AFTER INSERT ON [issues] BEGIN
  INSERT INTO [issues_fts] (rowid, [id], [content]) VALUES (new.rowid, new.[id], new.[content]);
END;
CREATE TRIGGER [issues_ad] AFTER DELETE ON [issues] BEGIN
  INSERT INTO [issues_

The `pages_metadata` column needs to be forced to an integer:

In [53]:
db_full["pages_metadata"].transform(types={"page_num": int})

<Table pages_metadata (id, page_idx, page_char_start, page_char_end, page_leaf_num, page_num, page_num_conf, page_text)>

## Search Over Metadata

In [3]:
from pandas import read_sql

read_sql("SELECT * FROM metadata LIMIT 3", db_full.conn)

,id,date,datetime,series,vol,iss,title,next_id,prev_id,is_index,restricted,year
0,sim_notes-and-queries_1849-11-03_1_1,1849-11-03,1849-11-03T00:00:00,None,1,1,Notes and Queries 1849-11-03: Vol 1 Iss 1,sim_notes-and-queries_1849-11-10_1_2,sim_notes-and-queries_1849-1850_1_index,0,,1849
1,sim_notes-and-queries_1849-11-10_1_2,1849-11-10,1849-11-10T00:00:00,None,1,2,Notes and Queries 1849-11-10: Vol 1 Iss 2,sim_notes-and-queries_1849-11-17_1_3,sim_notes-and-queries_1849-11-03_1_1,0,,1849
2,sim_notes-and-queries_1849-11-17_1_3,1849-11-17,1849-11-17T00:00:00,None,1,3,Notes and Queries 1849-11-17: Vol 1 Iss 3,sim_notes-and-queries_1849-11-24_1_4,sim_notes-and-queries_1849-11-10_1_2,0,,1849


### Modifying the `metadata` Table

Life you be so much easier if we had a `year` column, so let's modify the table to include one:

In [22]:
if not db_full["metadata"].exists():
    db_full["metadata"].add_column("year", int)

Now we can populate that via an upsert:

In [29]:
import pandas as pd

upserts = read_sql("SELECT id, datetime FROM metadata;", db_full.conn)

upserts['datetime'] = pd.to_datetime(upserts["datetime"])
upserts['year'] = upserts['datetime'].dt.year
upserts.head()

,id,datetime,year
0,sim_notes-and-queries_1849-11-03_1_1,1849-11-03,1849
1,sim_notes-and-queries_1849-11-10_1_2,1849-11-10,1849
2,sim_notes-and-queries_1849-11-17_1_3,1849-11-17,1849
3,sim_notes-and-queries_1849-11-24_1_4,1849-11-24,1849
4,sim_notes-and-queries_1849-12-01_1_5,1849-12-01,1849


In [31]:
#Do the upsert
db_full["metadata"].upsert_all(upserts[["id", "year"]].to_dict(orient="records"), pk=["id"])

<Table metadata (id, date, datetime, series, vol, iss, title, next_id, prev_id, is_index, restricted, year)>

## Search Over Issues Examples

In [12]:
read_sql("SELECT * FROM issues LIMIT 3", db_full.conn)

,id,content
0,sim_notes-and-queries_1849-11-03_1_1,\n \n \n \nNOTES anp QUERIES:\nA Medium of En...
1,sim_notes-and-queries_1849-11-10_1_2,|\n \nA MEDIUM OF INTER-COMMUNICATION\nFOR\nLI...
2,sim_notes-and-queries_1849-11-17_1_3,\n \n \nceeeeeeeeee eee\nA MEDIUM OF\nLITERAR...


## Pages Search Examples

In [3]:
read_sql("SELECT * FROM pages_metadata LIMIT 3", db_full.conn)

,id,page_idx,page_char_start,page_char_end,page_leaf_num,page_num,page_num_conf,page_text
0,sim_notes-and-queries_1849-11-03_1_1,0,0,301,1,,0.0,NOTES anp QUERIES:\nA Medium of Enter-Communic...
1,sim_notes-and-queries_1849-11-03_1_1,1,301,307,2,,0.0,
2,sim_notes-and-queries_1849-11-03_1_1,2,307,3212,3,2,100.0,NOTES ann QUERIES:\nA MEDIUM OF\n——_——s\nINTER...


In [253]:
search_term = 'Dilamgerbendi'
search_term = '"soul cake"'
#q = f"""
#SELECT * FROM pages_metadata_fts
#WHERE pages_metadata_fts MATCH {db.quote(search_term)};
#"""

q = """
SELECT id, snippet(pages_metadata_fts, -1, "__", "__", "...", 10) as clip
FROM pages_metadata_fts WHERE pages_metadata_fts MATCH {query};
"""

read_sql(q.format(query=db_full.quote(search_term)), db_full.conn)

,id,clip
0,sim_notes-and-queries_1851-11-15_4_107,"...Soul! soul! for a __soul-cake__ ; Pray, goo..."
1,sim_notes-and-queries_1868-12-12_2_50,...of which they call “ __soul-cakes__” or “so...
2,sim_notes-and-queries_1869-06-26_3_78,"...of that country, gathered __soul-cakes__ th..."
3,sim_notes-and-queries_1893-07-15_4_81,"...Assertions, 48 — Irish Cathedrals—** __Soul..."
4,sim_notes-and-queries_1893-07-15_4_81,...It was called “ __soul- caking__.” Was this...
5,sim_notes-and-queries_1893-08-05_4_84,...Misuse of Scientific Terms—* __Soul-caking_...
6,sim_notes-and-queries_1893-08-05_4_84,...49).—Your corre- spondent will find much in...
7,sim_notes-and-queries_1893-12-30_4_105,"...Aldgate or Aldersgate, 97 __Soul-caking__, ..."
8,sim_notes-and-queries_1893-12-30_4_105,"...a-snee, 13:3 __Soul-caking__, 117 Spurn-poi..."
9,sim_notes-and-queries_1893-12-30_4_105,"...Sombreset,” 154 Shakspeariana, 83 __Soul-ca..."


In [64]:
q = """
SELECT *
FROM pages_metadata WHERE page_text LIKE "%Isle of Wight%dedications%" LIMIT 3 ;
"""

zz=read_sql(q.format(query=db_full.quote(search_term)), db_full.conn)
zz


,id,page_idx,page_char_start,page_char_end,page_leaf_num,page_num,page_num_conf,page_text
0,sim_notes-and-queries_1857-02-14_3_59,4,20665,25191,5,126,100.0,"gnd S, No 59., Fes. 14. °57.]\n \ntine Empire,..."
1,sim_notes-and-queries_1857-02-28_3_61,17,93441,98769,18,179,100.0,"178\nNOTES AND QUERIES.\n[2e4 §, Ne 61, Fen. 2..."
2,sim_notes-and-queries_1857-02-28_3_61,19,105003,110416,20,181,0.0,"178\nNOTES AND QUERIES.\n[2=4 S. Ne 61., Fes. ..."


In [68]:
zz['page_text'][1]

"178\nNOTES AND QUERIES.\n[2e4 §, Ne 61, Fen. 28.°57.\n \nPanyer Alley before that of the carved comme-\nmorative tablet, which bears the date, I think, |\nAug. 27, 1688. Henry W. S. Tartror.\nSouthampton.\nDedications of the Isle of Wight Churches (2™S. iii. 125.) —I find the following dedications in Worsley’s History of the Isle of Wight: they fill up all the gaps in R. J. Jones's list of the ancient churches of the island, except those of Kingston, Shalfleet, Whippingham, and Yaverland :\nArreton. St. George.\nBrixton. St. Mary.\nCalborn. All Saints.\nGatcombe. St. Olave.\nGedshill. All Saints.\nMottiston. St. Peter and St. Paul. Newchurch. All Saints. Newport. St. Thomas Becket. Northwood. St. John Baptist Shorwell. St. Peter, not St. Paul. St. Helens. St. Helena.\nThorley. St. Swithin.\nWootton. St. Edmund.\nYarmouth. St. James.\nWorsley’s History was published in 1781. . 4 W.\n \n \n“ The Vicar and Moses” (2™ S. iii. 112.) — | Where the old and probably original impression |\nof 

In [65]:
zz['page_text'][2]

'178\nNOTES AND QUERIES.\n[2=4 S. Ne 61., Fes. 28. "57.\n \nPanyer Alley before that of the carved comme- morative tablet, which bears the date, I think, Aug. 27, 1688. Henry W. S. Tayrtor.\nSouthampton.\nDedications of the Isle of Wight Churches (2™S. iii. 125.) —I find the following dedications in Worsley’s History of the Isle of Wight: they fill up all the gaps in R. J. Jones\'s list of the ancient churches of the island, except those of Kingston, Shalfleet, Whippingham, and Yaverland :\nArreton. St. George.\nBrixton. St. Mary.\nCalborn. Alli Saints.\nGatcombe. St. Olave.\nGodshill. All Saints.\nMottiston. St. Peter and St. Paul. Newchurch. All Saints. Newport. St. Thomas Becket. Northwood. St. John Baptist Shorwell. St. Peter, not St. Paul. St. Helens. St. Helena.\nThorley. St. Swithin.\nWootton. St. Edmund.\nYarmouth. St. James.\nWorsley’s History was published in 1781. W. T. “ The Vicar and Moses” (2™ S. iii. 112.) — Where the old and probably original impression of this remarkab

## Index Search

We can perform a full text search on the terms found in the indexes.

In [6]:
read_sql("SELECT * FROM index_entries LIMIT 3", db_full.conn)[:50]

,source_id,year,vol,index_term,typ,subsidiary,page_num
0,sim_notes-and-queries_1849-1850_1_index,1849 - 1850,1,A.,P,None,NaN
1,sim_notes-and-queries_1849-1850_1_index,1849 - 1850,1,Apsey of St. Wandrille,S,,382.0
2,sim_notes-and-queries_1849-1850_1_index,1849 - 1850,1,Apsey of St. Wandrille,S,,486.0


And perform a full text search:

In [ ]:
Tje 

In [56]:
search_term = '"isle of wight"'

q = f"""
SELECT * FROM index_entries_fts
WHERE index_entries_fts MATCH {db_full.quote(search_term)} LIMIT 10;
"""

read_sql(q, db_full.conn)

,source_id,index_term,subsidiary,year,vol,page_num
0,sim_notes-and-queries_1855_12_index,Duppa (Bp.),"ug Serle (Wm.), of the Isle of Wight,",1855,12,224
1,sim_notes-and-queries_1857_3_index,Binstead,"Isle of Wight, inscriptions at,",1857,3,284
2,sim_notes-and-queries_1857_3_index,Isle of Wight church dedicatiuns,,1857,3,125
3,sim_notes-and-queries_1857_3_index,Isle of Wight church dedicatiuns,,1857,3,178
4,sim_notes-and-queries_1857_3_index,Isle of Wight church dedicatiuns,,1857,3,520
5,sim_notes-and-queries_1857_3_index,Jonas (R. J.),"on dedications of Isle of Wight churches,",1857,3,125
6,sim_notes-and-queries_1857_3_index,Dedications of Isle of Wight churches,,1857,3,520
7,sim_notes-and-queries_1857_3_index,Dedication of Isle of Wight churches,,1857,3,178
8,sim_notes-and-queries_1858_5_index,Brading,"Isle of Wight, its relics and customs,",1858,5,46
9,sim_notes-and-queries_1858_5_index,Isle of Wight,"its farm-houses,",1858,5,68


In [246]:
search_term = '"soul cake"'

q = f"""
SELECT * FROM index_entries_fts
WHERE index_entries_fts MATCH {db_full.quote(search_term)} LIMIT 10;
"""

read_sql(q, db_full.conn)

,source_id,index_term,subsidiary,year,vol,page_num
0,sim_notes-and-queries_1878_10_index,Soul cakes,,1878,10,426.0
1,sim_notes-and-queries_1878_10_index,Soul cakes,,1878,10,426.0
2,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,117.0
3,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,117.0
4,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,117.0
5,sim_notes-and-queries_1893_4_index,Soul-caking,custom,1893,4,NaN
6,sim_notes-and-queries_1893_4_index,Soul-caking,"on All Saints’ Day,",1893,4,49.0
7,sim_notes-and-queries_1893_4_index,Soul-caking,"on All Saints’ Day,",1893,4,117.0
8,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,49.0


## JOINed Search

Let's use the index to find some terms and then get the pages from the `pages_metadata` table.

??BORKED; or is there an off by one error??

*Not that this is quite  brittle search because it relies on the correct recognition of a page number from the OCR'd index text. However, we might be able to start correcting for that by checking references against the full text...*

In [256]:
search_term="soul cake"

q = f"""
SELECT p.*
FROM index_entries_fts AS ifts, metadata AS m,  pages_metadata AS p
WHERE index_entries_fts MATCH {db_full.quote(search_term)} AND
    ifts.year = m.year AND m.vol = ifts.vol AND p.id=m.id  AND (ifts.page_num)=p.page_num
LIMIT 10;
"""

#Off by 1 error
q2 = f""" SELECT * FROM (SELECT * FROM index_entries_fts
WHERE index_entries_fts MATCH {db_full.quote(search_term)} LIMIT 10) AS ifts, 
 metadata AS m,  pages_metadata AS p WHERE ifts.year = m.year AND m.vol = ifts.vol 
 AND p.id=m.id  AND ifts.page_num+1=p.page_num

"""
xx = read_sql(q2, db_full.conn)[:50]

xx

,source_id,index_term,subsidiary,year,vol,page_num,id,date,datetime,series,...,restricted,year,id,page_idx,page_char_start,page_char_end,page_leaf_num,page_num,page_num_conf,page_text
0,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,49,sim_notes-and-queries_1893-07-15_4_81,1893-07-15,1893-07-15T00:00:00,None,...,,1893,sim_notes-and-queries_1893-07-15_4_81,10,52281,58163,11,50,100.0,"r 15, '93.\nsaltire or, the first, shields sa...."
1,sim_notes-and-queries_1893_4_index,Soul-caking,"on All Saints’ Day,",1893,4,49,sim_notes-and-queries_1893-07-15_4_81,1893-07-15,1893-07-15T00:00:00,None,...,,1893,sim_notes-and-queries_1893-07-15_4_81,10,52281,58163,11,50,100.0,"r 15, '93.\nsaltire or, the first, shields sa...."
2,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,117,sim_notes-and-queries_1893-08-05_4_84,1893-08-05,1893-08-05T00:00:00,None,...,,1893,sim_notes-and-queries_1893-08-05_4_84,18,100358,106379,19,118,100.0,3.\nme\nrds\nrd-\naitle » it and rly Ow ng.\nw...
3,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,117,sim_notes-and-queries_1893-08-05_4_84,1893-08-05,1893-08-05T00:00:00,None,...,,1893,sim_notes-and-queries_1893-08-05_4_84,18,100358,106379,19,118,100.0,3.\nme\nrds\nrd-\naitle » it and rly Ow ng.\nw...
4,sim_notes-and-queries_1893_4_index,Soul-caking,,1893,4,117,sim_notes-and-queries_1893-08-05_4_84,1893-08-05,1893-08-05T00:00:00,None,...,,1893,sim_notes-and-queries_1893-08-05_4_84,18,100358,106379,19,118,100.0,3.\nme\nrds\nrd-\naitle » it and rly Ow ng.\nw...
5,sim_notes-and-queries_1893_4_index,Soul-caking,"on All Saints’ Day,",1893,4,117,sim_notes-and-queries_1893-08-05_4_84,1893-08-05,1893-08-05T00:00:00,None,...,,1893,sim_notes-and-queries_1893-08-05_4_84,18,100358,106379,19,118,100.0,3.\nme\nrds\nrd-\naitle » it and rly Ow ng.\nw...


In [259]:
xx['page_text'][3]

'3.\nme\nrds\nrd-\naitle » it and rly Ow ng.\nwW-\nill-\nrds mic ous an- on- | to the vas filo\nshe\nne\ned\n \ngh ¥, IV. Ave, 5, \'93.]\nNOTES AND QUERIES. 117\n \nMisuse or Scientiric Terms (8 S. iii. 286,\n436).—An amusing example of the confusion some- times caused by the use of sphere in its derived sense of ‘‘ province or duty ” occurs to my memory. The late Bishop of Ely, Dr. Woodford, was, like myself, a scholar of Pembroke College, Cambridge. A former master of that college, the once famous Lowndean Professor, Roger Long, celebrated in his day for his mechanical inventiveness and astrono- mical knowledge, constructed a hollow copper\nsphere representing the celestial firmament, in |\nwhich the constellations were shown by holes drilled according to their magnitude. It was large enough to stand or sit in, and was made to revolve by a winch. This sphere Dr. Long bequeathed to the college, and left a small sum to be paid annually to an undergraduate to exhibit it when called on. 

In [5]:
search_term = '"Tales and Legends of the Isle of Wight"'

#q = f"""
#SELECT * FROM pages_metadata_fts
#WHERE pages_metadata_fts MATCH {db.quote(search_term)};
#"""

q = """
SELECT id, snippet(pages_metadata_fts, -1, "__", "__", "...", 10) as clip
FROM pages_metadata_fts WHERE pages_metadata_fts MATCH {query} LIMIT 10;
"""

read_sql(q.format(query=db_full.quote(search_term)), db_full.conn)

,id,clip
0,sim_notes-and-queries_1873-08-30_12_296,"...Langley, York — Royalist Rising in Kent (16..."
1,sim_notes-and-queries_1873-12-27_12_313,...267 __Tales and Legends of the Isle of Wigh...
2,sim_notes-and-queries_1873-12-27_12_313,"...Cuthbert’s burial-place, 438 “* __Tales and..."
3,sim_notes-and-queries_1876-10-21_6_147,...In __Tales and Legends of the Isle of Wight...


In [72]:
search_term = '"sin drinking"'

#q = f"""
#SELECT * FROM pages_metadata_fts
#WHERE pages_metadata_fts MATCH {db.quote(search_term)};
#"""

q = """
SELECT id, snippet(pages_metadata_fts, -1, "__", "__", "...", 10) as clip
FROM pages_metadata_fts WHERE pages_metadata_fts MATCH {query} LIMIT 10;
"""

read_sql(q.format(query=db_full.quote(search_term)), db_full.conn)

,id,clip
0,sim_notes-and-queries_1896-02-29_9_218,...eating but also with __sin-drinking__. It m...
1,sim_notes-and-queries_1896-03-21_9_221,...1) if the term “‘__sin-drinking__” was empl...
2,sim_notes-and-queries_1896-04-11_9_224,...specifically. (1) The term ‘‘__sin- drinkin...


## Parsing Lists

In [202]:
items = """
https://archive.org/details/sim_notes-and-queries_1876-02-12_5_111/page/124/mode/2up Notes and Queries  1876-02-12: Vol 5 Iss 111 p124
https://archive.org/details/sim_notes-and-queries_1876-04-08_5_119/page/294/mode/2up Notes and Queries  1876-04-08: Vol 5 Iss 119 p294
https://archive.org/details/sim_notes-and-queries_1876-06-03_5_127/page/456/mode/2up Notes and Queries  1876-06-03: Vol 5 Iss 127 p457
https://archive.org/details/sim_notes-and-queries_1876-07-01_6_131/page/12/mode/2up Notes and Queries  1876-07-01: Vol 6 Iss 131 p12
https://archive.org/details/sim_notes-and-queries_1876-07-08_6_132/page/36/mode/2up Notes and Queries  1876-07-08: Vol 6 Iss 132 p37
https://archive.org/details/sim_notes-and-queries_1876-07-29_6_135/page/90/mode/2up Notes and Queries  1876-07-29: Vol 6 Iss 135 p90-1
https://archive.org/details/sim_notes-and-queries_1876-08-12_6_137/page/136/mode/2up Notes and Queries  1876-08-12: Vol 6 Iss 137 p137-8
https://archive.org/details/sim_notes-and-queries_1876-09-02_6_140/page/198/mode/2up Notes and Queries  1876-09-02: Vol 6 Iss 140 p198
https://archive.org/details/sim_notes-and-queries_1877-01-27_7_161/page/76/mode/2up Notes and Queries  1877-01-27: Vol 7 Iss 161 p77
https://archive.org/details/sim_notes-and-queries_1877-05-12_7_176/page/374/mode/2up Notes and Queries  1877-05-12: Vol 7 Iss 176 p375
https://archive.org/details/sim_notes-and-queries_1877-06-23_7_182/page/n15/mode/2up Notes and Queries  1877-06-23: Vol 7 Iss 182 p497
https://archive.org/details/sim_notes-and-queries_1877-10-13_8_198/page/298/mode/2up Notes and Queries  1877-10-13: Vol 8 Iss 198 p298
https://archive.org/details/sim_notes-and-queries_1877-12-01_8_205/page Notes and Queries  1877-12-01: Vol 8 Iss 205 p421
https://archive.org/details/sim_notes-and-queries_1878-01-12_9_211/page/n17/mode/2up Notes and Queries  1878-01-12: Vol 9 Iss 211 p39
"""

In [203]:
from parse import parse

parsed = []

for item in items.split("\n"):
    if item:
        _parsed = parse("https://archive.org/details/{id}/{} Notes and Queries{:s}{date}: Vol {vol} Iss {iss} p{p}", item)
        parsed.append(_parsed.__dict__['named'])

parsed

[{'id': 'sim_notes-and-queries_1876-02-12_5_111',
  'date': '1876-02-12',
  'vol': '5',
  'iss': '111',
  'p': '124'},
 {'id': 'sim_notes-and-queries_1876-04-08_5_119',
  'date': '1876-04-08',
  'vol': '5',
  'iss': '119',
  'p': '294'},
 {'id': 'sim_notes-and-queries_1876-06-03_5_127',
  'date': '1876-06-03',
  'vol': '5',
  'iss': '127',
  'p': '457'},
 {'id': 'sim_notes-and-queries_1876-07-01_6_131',
  'date': '1876-07-01',
  'vol': '6',
  'iss': '131',
  'p': '12'},
 {'id': 'sim_notes-and-queries_1876-07-08_6_132',
  'date': '1876-07-08',
  'vol': '6',
  'iss': '132',
  'p': '37'},
 {'id': 'sim_notes-and-queries_1876-07-29_6_135',
  'date': '1876-07-29',
  'vol': '6',
  'iss': '135',
  'p': '90-1'},
 {'id': 'sim_notes-and-queries_1876-08-12_6_137',
  'date': '1876-08-12',
  'vol': '6',
  'iss': '137',
  'p': '137-8'},
 {'id': 'sim_notes-and-queries_1876-09-02_6_140',
  'date': '1876-09-02',
  'vol': '6',
  'iss': '140',
  'p': '198'},
 {'id': 'sim_notes-and-queries_1877-01-27_7_161

If there is a page range provided, we need to parse the range:

In [205]:
def get_range(pn):
    """Get range."""
    _pn = str(pn).split('-')
    return (int(_pn[0]), int(_pn[0])) if len(_pn)==1 \
        else (int(_pn[0]), int(_pn[0][:-len(_pn[1])]+_pn[1]))

Let's just test that:

In [206]:
for pn in [11, "91-92", "1-10", "21-31", "91-4", "123-4", "123-32", '123-145']:
    print(pn, get_range(pn))

11 (11, 11)
91-92 (91, 92)
1-10 (1, 10)
21-31 (21, 31)
91-4 (91, 94)
123-4 (123, 124)
123-32 (123, 132)
123-145 (123, 145)


Extract the page ranges for our references:

In [209]:
for p in parsed:
    (p['from'], p['to']) = get_range(p['p'])
    
parsed

[{'id': 'sim_notes-and-queries_1876-02-12_5_111',
  'date': '1876-02-12',
  'vol': '5',
  'iss': '111',
  'p': '124',
  'from': 124,
  'to': 124},
 {'id': 'sim_notes-and-queries_1876-04-08_5_119',
  'date': '1876-04-08',
  'vol': '5',
  'iss': '119',
  'p': '294',
  'from': 294,
  'to': 294},
 {'id': 'sim_notes-and-queries_1876-06-03_5_127',
  'date': '1876-06-03',
  'vol': '5',
  'iss': '127',
  'p': '457',
  'from': 457,
  'to': 457},
 {'id': 'sim_notes-and-queries_1876-07-01_6_131',
  'date': '1876-07-01',
  'vol': '6',
  'iss': '131',
  'p': '12',
  'from': 12,
  'to': 12},
 {'id': 'sim_notes-and-queries_1876-07-08_6_132',
  'date': '1876-07-08',
  'vol': '6',
  'iss': '132',
  'p': '37',
  'from': 37,
  'to': 37},
 {'id': 'sim_notes-and-queries_1876-07-29_6_135',
  'date': '1876-07-29',
  'vol': '6',
  'iss': '135',
  'p': '90-1',
  'from': 90,
  'to': 91},
 {'id': 'sim_notes-and-queries_1876-08-12_6_137',
  'date': '1876-08-12',
  'vol': '6',
  'iss': '137',
  'p': '137-8',
  'fr

Let's have a go at grabbing the pages...

In [218]:
_pages = []
_not_resolved = []
for record in parsed:
    _id = record["id"]
    for _p in range(record["from"], record["to"]+1):
        # ??  off by one error?
        q = f'SELECT page_text FROM pages_metadata WHERE id="{_id}" AND page_num={_p+1}'
        try:
            _pages.append(f'{record["id"]} - {record["date"]}, vol {record["vol"]} iss {record["iss"]} p.{record["p"]}\n\n'+ \
                      read_sql(q, db_full.conn).loc[0,'page_text'] +"\n\n---\n\n" )
        except:
            _not_resolved.append(record)


In [219]:
# Eg may not be resolved be no page numbers
_not_resolved

[{'id': 'sim_notes-and-queries_1877-06-23_7_182',
  'date': '1877-06-23',
  'vol': '7',
  'iss': '182',
  'p': '497',
  'from': 497,
  'to': 497}]

In [220]:
from IPython.display import Markdown

Markdown(_pages[4])

sim_notes-and-queries_1876-07-08_6_132 - 1876-07-08, vol 6 iss 132 p.37

> have | from
entury, maker ex hibi- » at his lerlin’s
om pass in the
 
une irde ses 3 yn
 
  
me sus- isten to pon in- vurage in my with an journal signed yon the design age the es from pounds, 4] which smen of
|
tin ] », @ » Epis
 
5* 8. VI. Jury 8, '76.]
NOTES AND QUERIES. 37
 
tolaris ype: Davide Hartley, A.M. e R.S.S., to which he added Conjecture om de Sensu, Motu, et Idearum Generatione. Editio secunda, Bathonie, 1746. As he lived till August, 1757, we may suppose that the benefit he derived from the medicine was permanent.
“ The concurring testimony of all the best authorities of the time gives us assurance of the signal benefit that was often derived from the use of Stephens’s......remedy, which consisted especially of a mixture of calcined egg- shells and Castile soap.”— British and Fore gine Medical Review, vol. xii. p. 391.
D. D.
(See “N. & Q.,” 1" S. xii. 366; 4" S. ix. 84]
A Fotx-Lore Soctery (5™ §, v. 124, 294, 457 vi. 12..—Mr. Rarcuirre has mistaken (p. 457 the name of the newspaper in this district which makes “antiquarian notes,” &c., “a princ mr feature in its weekly issue.” It is the Newcastle Weekly Chronicle, and not the paper he mentions, that Mr. Rarciirre must have had in his mind. The “ Notes and (Queries > of the Weekly Chroniel were commenced about three years ago.
Ernest WELLS.
Besides pm
Newcastle-on-Tyne.
The Glasgow Weekly Herald has a correspon- dents’ column, wherein not infreqt lore crop up among the Memoranda.” I thor ath agree with your cor- respondents that Folk-Lore Society is now be- come almost a necessity, and hope = Moodie vay that the project may not be allowed t ground, but that we may very energetic and useful Society. A great field li still open for cultivation ; and I have little dou rt but that when the scheme is thoroughly ventilated, there will be no lack of the needful m¢ mbers to support and carry out its aims. The proposal of St. SWITHIN is an opportune one, and with able
*(Juestions, Answers, and
soon have an
e bt
men to guide the Society we need not fear any failure. Wituram Georce Brack. “ GonE JericHo” (2™4 §S. ii. 330, 395; 5%
8S. v. 415, 474.)—In the parish of Stapleford,
Leicestershire, is a farmhouse called Jericho Lodge ; and a portion of land near is marked on the Ordnance ma p as Jericho pastures. A stream
passing through the pastures divides Leicester- shire from Rutlandshire. origin of the name in this instance. person in Jericho is not an uncommon form of objurgation in Leicestershire.
Tuomas Norra.
Wuirrine Docs ovt or Cuurcn (5" §, 309, 514; v. 37, 136, 419.)\—In Baslow C hureh | an ancient chapel of Bakewell, Derbyshire, there is still preserved the impleme nt of a do g-whipper. There are also persons yet alive, or only recently
edeceased, who can recollect its use. The tho ng of |
1ently bits of folk- |
to fall to the |
I am unable to give the To wish a |
| |
the whip is about three feet long, and is fastened to a short ash stick, round the handle of which is a band of twisted leather. I do not think that wonderful instrument in the church of Clynnog- Fawr, North Wales, for dragging dogs out of church has ever been noticed in “N. & Q.” It is 2 long pair of iron “lazy-tongs,” with short spikes fixed at the ends. J. Cuarues Cox,
 
| Tt appears from the churchwardens’ accounts of | the parish of Worksop, from which a few items | are copied in White’s Worksop, the Dukery, and | Sherwood Forest, 1875, that in 1597 the sum of 9d. was “ paid to old Verde for whipping of dogs,” and “ for whipping dogges out of ye church one whole year” the sum of 12d. was paid in 1616.
J. Porrer Briscoe.
Nottingham.
Mrsister: Priest (5@ §. v. 449, 494.)—I possess a small Common Prayer Book, “ Imprinted |by Rob. Barker,” 1638. The rubric in the | Morning Prayer runs, “The absolution or remission | of sins, to be pronounced by the priest alone.” | In the “ Visitation of the Sick” the first four | paragraphs of the rubric minister, | the fifth and last ordains that “after confession | the priest shall absolve him after this sort, a J. O.
 
 
speak ot the
| Ihave a copy of the Book of Common Prayer, | . ‘Pp rin te ~d by Thom Buc k and R wer Daniel, | printers to the Universitie of Cambridge,” 4to., | 1637, and the Absolution in the Morning Prayer is to be pronounced by the “ Minister.” In the Communion Service the rubric before the General Confession runs as follows: “Then shall this general confession be made in th ume r: all | those that are minded to receive the Holy Com- munion, either b y one of them, or el by one of
the minist ers, or by the priest himself, all kneel- ig,” &e. Do the words “ either by one of them” occur in any other edition ?
? This edition of 1637 I believe,
 
    
lI
scarce, not bein ng in the Lea Wilson and tion.
A. BATEMAN. Randolph Gardens, W.
The rubric to the Absolution in the Book of
Common Prayer of 1637 says :—
“The absolution or remission of sinnes to be pro- nounced by the Presbyter alone, he standing up, and turning himself to the people, but they still remaining humbly upon their knees.”
M.. ¥.
In answer to Mr. Dore, the only Prayer Book I have between 1636 and 16:
“London: Printed by Robert Barker, Printer to the King’s most excellent Majesty: and by the Assignes of John Bill. 1638.” 12mo.
In this, the term rubric prefixed to the
39 is—
 
“Priest” is used in the
Absolution. _

---



In [221]:
Markdown("".join(_pages))

sim_notes-and-queries_1876-02-12_5_111 - 1876-02-12, vol 5 iss 111 p.124

124 NOTES AND QUERIES.
 
[5 8. V. Fes. 12, °76,
 
H. 8. King & Co., 1874), may be fitly presented to the readers of “N. & Q.” :—
“Some of my readers will feel disposed to laugh out- right at a learned doctor admitting he is an inveterate believer in all sorts of popular superstitions, forebodings, and presentiments. I am alarmed at the spilling of a salt-cellar; I don’t like to meet a hearse while going out of the street door ; I would not undertake a journey or any important work on a Friday; and the breaking of a looking-glass would throw me into fits. Now this morn- ing,* soon after our téte-d-téte déjeiiner, I became sud- denly depressed in spirits, to such a degree that my fair hostess fancied I had been taken ill. This state of nervousness and depression endured after I had retired to my hotel, and was making ready my luggage for my positive departure at noon on the succeeding day, leaving out only the evening dress for the dinner and opera. On taking my place at dinner the knife and fork laid before me crossway startled me (I dare say I turned pale), but I said nothing. There were two attendants. At the next course the other valet replaced my plate, and again the fatal cross was laid before me! I looked round to the three guests to see if it was the habit of the servants of the house; they had no cross, only the doctor : and again the third time the same symbol made its appearance before me with the setting of the dessert and corresponding plates with gilt knife and fork, the two latter “of whi ch articles again contrived to be laid down in a crucial form. Ah! now there was no mistake. Some great crossing was about to befall me. I had bet- ter shut myself up for the rest of the day, give up the proposed drive and the opera, and wait until I can escape in the morning from the doomed city. To make matters still more formidable, I found, on looking at my calendar, that it was Friday. All this mind-work I of course kept to myself, albeit I must have appeared rather more stupid than was my wont.”—Vol. i. p. 413,
y hours later Dr. Granville was arrests uch for prophetic cutlery. St. SwitHrn
‘oLtK-Lore Socrety.—I am not alone in g it high time that steps should be taken a society for collecting, arranging, and all the scattered bits of folk-lore which r ‘a of in books and hear of in the flesh. Such «% society should not confine its labours to the lk-lore of our own land, but should have members and workers everywhere. St. SwirHr.
 
THE FRENCH STATE PAPER OFFICE. Histoire du Dépét des Archives des Affaires Etrangéres Paris au Louvre en 1710, a Ver saille s en 1763, at de
nouvean « Paris en Divers Endroits depuis 1796.. Par Armand Baschet. Svo. Paris, Plon. Concluding Article. ) We now come to the last division of M. Baschet’s work, including the space of time which has elapsed between 1796 and 1853. A number of well-known names meet us at almost every step in this interesting gallery of portraits : Anquetil, Soulavie, Lemontey, Bignon, and Saint-Priest, to say nothing of the keepers of the Record Office
* On a day in 1814, when the Doctor was at Bologna, and the guest of Medame Martinetti.
themselves, men who have obtained and deserved European celebrity.
Count d’Hauterive must first be mentioned here. A friend of Talleyrand, courteous and obliging in private life, parfait gentilhomme, but doggedly resolved upon keeping the Foreign Office papers unsullied by the hands of historical stu- dents, he would have deemed the communication of the treaty between Karl the Bald and Ludwig the German fraught with danger to the state. It was during Count d’Hauterive’s administration that Lemontey and Sir James Mackintosh obtained permission to make transcripts from the documents preserved in the Dépét des Relations Extérieures, the former for the history of the reign of Louis XV., the latter for his account of the Revolution of 1688. The dragon who watched with such jealous care the diplomatic garden of the Hesperides was obliged to yield ; but, if he could not wreak his vengeance upon the English statesman, he made, at any rate, Lemontey’s heirs smart for it. Immediately after the historian’s death, all his papers, notes,and MSS, were seized and confiscated, and the Gazette des Tribunaux for August, 1826, contains the details of the lawsuit which re esulted from that unwarrant-
 
 
 
able act of administrative caprice.
To Count d =e succeeded, in 1830, M. Mignet, the present distinguished secretary of the Académie des Sciences Moral et P ritiques. Thanks to M. Guizot’s initiative, a vig oro m0 $] impetus had been given to historical studies, and M. Baschet takes the op pportt inity of : ippreciat ting in detail the
results of this species of scientific revival, paying at the same time a tribute of just praise to the learned editor of the Ne a tions relatives & la Nuce sso? ad’ Bs ;
I shall say nothing of Messrs. Carteron and Cintrat, who occupied successively the post which M. Mignet left vacant when the Revolution of 1848 broke out. M. Prosper Faugére, on the other hand, deserves a distinct notice; and I wish I could translate in extenso M. Baschet’s biographical sketch of this excellent French scholar. But I must forbear, and remain satisfied with alluding to the controversy which broke out between him and M. Victor Cousin on the subject of Pascal's Pensées, the latter not only maintaining that the austere Port-Royalist was in love, but trying to find out what lady had succeeded in winning his affec- tion; the former indignantly protesting against the cynicism which transformed Pascal into a kind of inamorato. These literary debates, however, have nothing to do with M. Faugére’s fitness for his post as Keeper of the Archives at the French Foreign Office. It would be impossible to imagine a better choice in ev rery respect ; and the ghost of poor D’Hauterive must feel dismayed indeed at the prospect of the liberal reforms which the Duke Decazes, supported by his able coadjutor, has sanctioned. Let me repeat that to M. Armand
 
  
B

---

sim_notes-and-queries_1876-04-08_5_119 - 1876-04-08, vol 5 iss 119 p.294

294
NOTES AND QUERIES.
(5" 8. V. Apris 8, '7
 
ages as to the date of Lord Ligonier’s marriage to Penelope Pitt. It is stated to be January 1, 1767, not May, 1766, as stated above in the Act. J. Horsratt TURNER. Idle, Leeds.
Rev. Taomas Harwarp (5" §, v. 249.)— Perhaps Cot. Fisnwick and I may be speaking of two different persons ; but, notwithstanding a serious discrepancy in the date of death, I suspect —from coincidence in date of birth, and of appoint- ment to a Lancashire living (Garstang), and identity of college and name of father, and some resemblance in sound between Warwick and Warrington—that he refers to the Rev. Thomas Hayward, one of the head masters of Boteler’s Free Grammar School at Warrington, of whom an account will be found in vol. viii., p. 69, of the Transactions of the Historic Society of Lancashire and Cheshire, in a paper by me on the subject of that Foundation. I may add to the information there given, which is too lengthy for transfer to your columns, that his father, Thomas Hayward, who was an attorney at Warrington, was baptized at the parochial chapel of Daresbury, Cheshire, a few miles from Warrington ; he is stated in the register to have been the son of George Hayward, of Keiquicke (one of the many ways of spelling Keckwick, a township in that chapelry), and to have been born July 23, 1655. J. F. Marsn.
Hardwick House, Chepstow.
A Forx-Lore Socrery (5 §. v. 124.)—May I call the attention of St. Swirxin and others to the following suggestions? “The Decay of Rural Tradition ” is the title of an article in the Satur- day Review for Feb. 5, 1876, in which it is stated that the editor of a Worcestershire newspaper has invited co-operation throughout that county in an experiment to collect “the floating and unwritten, or at least unpublished, treasures of local tradi- tions, customs, legends, proverbs, rhymes, and miscellaneous fireside memories belonging to the county.” The idea is so excellent that I wish it every success. My object in referring to the sub- ject here is to hope that the same thing may be done in every county, and that the collections may include writings in various dialects, lists of locally- printed books (many of which never reach Blooms- bury), of all the works, small as well as great, relating to the county, of local worthies, and of authors, celebrated and comparatively unknown, born within the county. As copies of all our newspapers are preserved at the British Museum, the collections would soon prove of great value to the antiquary, and a thousand waifs and strays, that would not find admission even to the pages of “N. & Q.,” be preserved. The schoolmaster who laughs at old traditions, and the railway that is rapidly mixing the population of the country, are sad destroyers of our local legends. Who, then,
 
of newspaper editors will come to the rescue by opening their columns for this purpose, say, on one particular Saturday of the month? and who of clergymen, doctors, and antiquaries will take the trouble to seek for and write down such folk-lore as may yet be found lingering in our less- enlightened districts ? H. Bower.
Autrors WantTeD (5 §. v. 248.)—The corre- spondent who asks for the author of Modern Cha- racters from Shakspeare is probably not aware that the volume was a collection from some newspaper of the day, and, as it was published anonymously, there is little chance that the writer will ever be discovered.
The book would probably have been long since forgotten, had it not been for the fact of its being mentioned by Boswell in his Life of Johnson :—
“ This season (1778) there was a whimsical fashion in the newspapers (qy. did these characters appear in more than one!) of applying Shakspeare’s words to describe living persons well known in the world, which was done under the title of Modern Characters from Shakspeare, many of which were admirably adapted. The fancy took so much that they were afterwards collected into a pamphlet. Somebody said to Johnson, across the table, that he had not been in these characters. ‘ Yes,’ said he, ‘I have; I should have been sorry to have been left out.’ He then repeated what had been applied to him :—
‘ You must borrow me Garagantua’s mouth.’ Miss Reynolds not perceiving at once the meaning, he was obliged to explain it to her, which had something of an awkward and ludicrous effect : ‘Why, madam, it has reference to me as using big words, which require the mouth of a giant to pronounce them. Garagantua is the name of a giant in Rabelais.’”
The sentence given in the book is—
“ You must borrow me Garagantua’s mouth,
Tis a word too great for any mouth of this age’s size.’ If I remember rightly, it has been shown in these pages that Garaganta, not Garagantua, is the proper reading.
If guessing were permitted, I should be inclined to suggest that Caleb Whitefoord was the author, especially if the “Characters” appeared in the Public Advertiser, as “he turned and moulded the various topics of the day into all sorts of shapes” for that paper. An example may be named in the “ cross readings of the newspapers,” which were so popular that Goldsmith “ would gladly have exchanged for it his own most success- ful writings.” —Forster’s Life of Goldsmith, vol. ii. p. 46.
Modern Characters is mentioned by Mr. Halliwell in his Shakspeariana (1841), and is No. 68 in the list of “ Commentaries, Essays,” Xe.
CuarLes WYLIE.
The Economy of Human Life, by Robert Dods- ley, first appeared in 1751, and has been very often reprinted. “Gander, Gregory, Kt., Poetical Tales, Bath, 1779, sm. 4to., 1s.,” is all the information

---

sim_notes-and-queries_1876-06-03_5_127 - 1876-06-03, vol 5 iss 127 p.457

mai aeovw =e *
 
§* 8. V. Jone 3,’76.)
NOTES AND QUERIES.
   
457
 
the Earl of Essex. The likeness of Richard IT. seems, if my memory does not deceive me, to be due to the famous portrait of the monarch which is now, I believe, in Westminster Abbey, after an elegant “ restoration,” and supposed to be the sole authority for the subject, except coins. I resume Vertue referred to coins, and a vigorous inner consciousness, for portraits of the earlier sovereigns. F. G. 8.
AspInwALt 1x America (5® §. v. 9.)—This town was named after Mr. Aspinwall, of the firm of Howland & Aspinwall, proprietors of the line of steamers at the time running from New York to Aspinwall, and from Panama to San Francisco. The town of Colon, established in the English interests, has not prospered. The old Spanish town of Chagres, near the above, in Central America, or Granada, was badly situated on the river of the same name. J. McC. B.
Hobart Town.
“TrrtoTtaL” (5% §, iv. 429; v. 18, 137, 398.) —The account given by Mr. Kearton to Dr. Drxon was incorrect in every particular. 1. The person referred to was Mr. Swindlehurst, not “Swingelhurst”; 2. Mr. Swindlehurst was not a stutterer ; 3. He had nothing whatever to do with the origin of the word “ teetotal,” or its application to the total abstinence movement.
Dawson Burns.
Sr. Curnzert (5 §. v. 387.)—Can the learned J.T. F. tell me why donkeys are called cuddies in the north of England, after St. Cuthbert, and neddies in the south, after St. Edward ?
F. B.
Derivation oF Stiutox, Guarroy, AND Con- NINGTON (5 §. v. 109, 236.)—Stilton, from Anglo- Saxon Stael, a man’s name : Stael’s town. Glatton, from the British word glastennen, the holm-oak— 4.e. the town of the holm-oak. Connington has been already given. HIRONDELLE.
Walsall.
“Pretty ” (3% §. vii. 453 ; viii. 7, 57, 98, 137, 197 ; 5" S. v. 214, 276.)—The meaning here given to the word has been retained by a lower grade in society—cottagers—who cleave to the ancient usage of words for many years after their meanings or + pe mar have been changed by the middle and upper classes. It was only lately that I was giving some “picture books” to a little child, when her grandmother said to her, “Now, my dear, speak pretty to the gentleman.” I also hear from cottagers the phrase, “He behaved very pretty,” where the person in question had done what was expected of him, or shown some kind- ness. The gipsy fortune-teller’s “pretty gentle-
man” would seem to have the same signification. Curnpert Bepe.
 
 
“Tnroxicatine” (5% §. iv. 409, 523; v. 137, 276.)—The two following passages from Sir D. Lyndesay’s Works, pp. 415, 418 (E. E. T. Soc.), carry back the use of the word beyond Milton. The former passage has the sense “ poisoned,” but the latter has the modern meaning, or is closely akin to it :—
“ Kings sould of gude exempils be the well ;
Bot, gif that your strands be intoxicate,
In steid of wyne, thay drink the poyson fell.” “Ye se the king is yit effeminate,
And gydit be Dame Sensualitie,
Rycht sa with young counsall inforicate ;
Swa at this tyme ye haif your libertie.”
Go W..ae
A Forx-Lore Socrery (5" §. v. 124, 294.)— The suggestions of Mr. Bower at the latter refer- ence are being carried out, and have been worked with great success for some years, in the Midlands, and a great deal of matter of interest has been col- lected. Some six or seven years ago a Birming- ham newspaper (I forget which) began to collect old lore. This was followed by a Chesterfield paper, The Derbyshire Times, which, under the special editorship of a well-known contributor to “N. & Q.,” published weekly for several years from one to three cols. of valuable and interesting matter, under the heading of “ Local Notes and Queries.” This form of “news” was soon imitated by neighbouring papers, with pretty fair success on the whole. Amongst the papers which con- tinue to collect from their readers under this head- ing I may name the Manchester Guardian, the Sheffield and Rotherham Independent, the Brad- ford Observer, and three Derby papers—the Reporter, Advertiser, and Mercury. Several Che- shire newspapers publish antiquarian notes, and the North of England Advertiser a short time ago made this kind of reading a principal feature in its weekly issue. Tomas RatcLiFrFrE.
Worksop.
Domespay Boox (5* §. v. 188, 354.)—Ingul- phus cannot be used as evidence for what took place in the reign of Alfred. Great part of the book is now known to be a mere romance. I have not Carte at hand to see what he says concerning the supposed Domesday Book of Alfred. If such a record was ever compiled, which I doubt, it is highly improbable that it was in existence in the reign of King Edward IV. A. 0. V. ®
“Snampies” (5 §. v. 261, 355.)—In illustra- tion of the earlier use of this word, whose history has been so clearly traced by Mr. Skeat, reference may be made to the Authorized Version of 1 Cor. x. 25, “ Whatsoever is sold in the shambles, that eat, asking no question for conscience sake,” where the original reads év paxéAAw. This is an instance of a Latin word in Greek garb. Ma- cellum, whatever its etymology, signifies, like

---

sim_notes-and-queries_1876-07-01_6_131 - 1876-07-01, vol 6 iss 131 p.12

NOTES AND QUERIES.
 
(5° 8. VL. Jony 1, ’76.
 
of this constellation from northern latitudes is to be found in the precession of the equinoxes, or the slight receding westward of the points where the ecliptic or sun’s path crosses the equator twice a year. Owing to the greater thickness of the earth at the equator, that part of the earth comes to the equinoctial points a little sooner ; consequently the sun appears to recede or go back towards the west, and the North Pole moves every year a little back- ward on the circle it describes in the northern sky. This movement being about 50” of a degree yearly, the relative place of the stars in ancient times can be ascertained by it. The North Pole makes this circle in about 25,900 years. The North Polar star of to-day has not always been nor will it continue to be the Pole star. At the time of the construc- tion of the earliest catalogues of the stars, 120 z.c., it was 12° from the Pole, it is now only 1° 24’ ; it will continue to approach to within half a degree, and will then recede. At the time of the erection of the great Pyramid of Gizeh, some 4,000 years ago, the pole of the heavens was near Alpha Draconis. It is a curious fact that of the nine pyramids, six of the largest had the narrow entrance passages inclined downwards at such an angle that the Pole star of that time must have been visible, perhaps in daylight. In about 12,000 years the bright star Vega or Alpha Lyre will cose the Pole star. As regards the Star of Bethlehem, the star that guided the Wise Men, it is a matter of history that about 125 years B.c. a bright star appeared, and gradually increased in brilliancy, so as to be seen in the daytime about the time of our Saviour ; it gradually decreased in brightness and disappeared. It was the appear- ance of this star that induced Hipparchus to draw up his catalogue. It was situated in the constella- tion Coma or Koma, not far from Virgo. Its great peculiarity would be that its appearance had been predicted some 1,400 years before. From its position it would culminate, or be on the meridian about twelve at night, in the latitude of Jerusalem.
“ It is a fact independent of all hypotheses that at the precise hour of midnight, when Christ was born and Christianity appeared, the celestial sign which mounted above the horizon was Virgo.”—Dupuwis’s Orig. des Cultes.
R. C. Trench speaks of this star “shining in calm and silent splendour, larger, lévelier, and brighter than any of the host of heaven.”—R. C. Trench’s Star of the Wise Men.
He also quotes from Ignatius, Bishop of Antioch, who may have heard of it from those who had seen it. Prudentius is also quoted, “ that not even the star of the morning was fairer.” He also adds we have many allusions in the ancient Christian writers “to the surpassing brightness of this star,” which I conceive, as many ancients and moderns have done, to have been a new star in the heavens.
 
We have many records of temporary stars that have appeared and then vanished..
In the Zend Avesta it is said that Zerdust or Zoroaster, who taught and founded the sect of the Magi, informed his followers that a star should appear at the birth of the Messiah, or the promised and desired one, and that when it appeared they should go and offer gifts, and worship the great one. It would appear very probable that Zoro- aster was a Jew by birth anda disciple of Daniel’s, and acquainted with Daniel’s prophecy of seventy weeks of years, and with Balaam’s prophecy in Numbers.
A star in the Eastern nation was a sign of divine dignity. Christ calls himself the bright and morning star (Rev. xxii. 16).
See Bishop Horsley on the prophecies of the Messiah, Gill’s Commentary, Trench’s Star of the Wise Men. Wittiam HEane.
Cinderford.
A Foux-Lore Socrery (5" §. v. 124, 294, 457.) —Hailing as I do with great satisfaction the ex- cellent suggestion of your accomplished correspon- dent St. Swirnty (5" 8S. v. 124), I must admit that I have been greatly disappointed that that proposal has not elicited greater support from your readers. As one who suggested upwards of thirty years ago the advisability of collecting the remains of our popular mythology and superstitions before they were quite trampled out by the iron horse, and who has never ceased to take an interest in the subject, I venture to say that not a day should be lost in organizing such a society. A central committee in London, of some half dozen who have made Folk-Lore more or less a study, with local secretaries in different parts of the country, if backed by a couple of hundred subscribers of a guinea, might collect and print an interesting yearly volume.
Judging from what I have seen in some local newspapers, one of the committee’s greatest diffi- culties would be not that of collecting, but that of selecting what is not already recorded by Brand and his editors, Sir Henry Eliis and Mr. Halliwell- Phillipps. This would require to be done with considerable judgment, and with great care, so as not to discourage those who take the trouble to communicate what they believe to be not generally known.
Ladies should be specially invited to take part in the work, who, in their kindly ministrations in the cottages of their poorer neighbours, must often come across traces of old world customs and beliefs.
Though I have spoken of an annual volume, I would not wait for the completion of a volume, but keep alive interest in the Society by issuing a few sheets from time to time, as soon as sufficient materials of interest had been collected. Success to the Folk-Lore Society !
Aw Oxtp Forx-Lorist.

---

sim_notes-and-queries_1876-07-08_6_132 - 1876-07-08, vol 6 iss 132 p.37

> have | from
entury, maker ex hibi- » at his lerlin’s
om pass in the
 
une irde ses 3 yn
 
  
me sus- isten to pon in- vurage in my with an journal signed yon the design age the es from pounds, 4] which smen of
|
tin ] », @ » Epis
 
5* 8. VI. Jury 8, '76.]
NOTES AND QUERIES. 37
 
tolaris ype: Davide Hartley, A.M. e R.S.S., to which he added Conjecture om de Sensu, Motu, et Idearum Generatione. Editio secunda, Bathonie, 1746. As he lived till August, 1757, we may suppose that the benefit he derived from the medicine was permanent.
“ The concurring testimony of all the best authorities of the time gives us assurance of the signal benefit that was often derived from the use of Stephens’s......remedy, which consisted especially of a mixture of calcined egg- shells and Castile soap.”— British and Fore gine Medical Review, vol. xii. p. 391.
D. D.
(See “N. & Q.,” 1" S. xii. 366; 4" S. ix. 84]
A Fotx-Lore Soctery (5™ §, v. 124, 294, 457 vi. 12..—Mr. Rarcuirre has mistaken (p. 457 the name of the newspaper in this district which makes “antiquarian notes,” &c., “a princ mr feature in its weekly issue.” It is the Newcastle Weekly Chronicle, and not the paper he mentions, that Mr. Rarciirre must have had in his mind. The “ Notes and (Queries > of the Weekly Chroniel were commenced about three years ago.
Ernest WELLS.
Besides pm
Newcastle-on-Tyne.
The Glasgow Weekly Herald has a correspon- dents’ column, wherein not infreqt lore crop up among the Memoranda.” I thor ath agree with your cor- respondents that Folk-Lore Society is now be- come almost a necessity, and hope = Moodie vay that the project may not be allowed t ground, but that we may very energetic and useful Society. A great field li still open for cultivation ; and I have little dou rt but that when the scheme is thoroughly ventilated, there will be no lack of the needful m¢ mbers to support and carry out its aims. The proposal of St. SWITHIN is an opportune one, and with able
*(Juestions, Answers, and
soon have an
e bt
men to guide the Society we need not fear any failure. Wituram Georce Brack. “ GonE JericHo” (2™4 §S. ii. 330, 395; 5%
8S. v. 415, 474.)—In the parish of Stapleford,
Leicestershire, is a farmhouse called Jericho Lodge ; and a portion of land near is marked on the Ordnance ma p as Jericho pastures. A stream
passing through the pastures divides Leicester- shire from Rutlandshire. origin of the name in this instance. person in Jericho is not an uncommon form of objurgation in Leicestershire.
Tuomas Norra.
Wuirrine Docs ovt or Cuurcn (5" §, 309, 514; v. 37, 136, 419.)\—In Baslow C hureh | an ancient chapel of Bakewell, Derbyshire, there is still preserved the impleme nt of a do g-whipper. There are also persons yet alive, or only recently
edeceased, who can recollect its use. The tho ng of |
1ently bits of folk- |
to fall to the |
I am unable to give the To wish a |
| |
the whip is about three feet long, and is fastened to a short ash stick, round the handle of which is a band of twisted leather. I do not think that wonderful instrument in the church of Clynnog- Fawr, North Wales, for dragging dogs out of church has ever been noticed in “N. & Q.” It is 2 long pair of iron “lazy-tongs,” with short spikes fixed at the ends. J. Cuarues Cox,
 
| Tt appears from the churchwardens’ accounts of | the parish of Worksop, from which a few items | are copied in White’s Worksop, the Dukery, and | Sherwood Forest, 1875, that in 1597 the sum of 9d. was “ paid to old Verde for whipping of dogs,” and “ for whipping dogges out of ye church one whole year” the sum of 12d. was paid in 1616.
J. Porrer Briscoe.
Nottingham.
Mrsister: Priest (5@ §. v. 449, 494.)—I possess a small Common Prayer Book, “ Imprinted |by Rob. Barker,” 1638. The rubric in the | Morning Prayer runs, “The absolution or remission | of sins, to be pronounced by the priest alone.” | In the “ Visitation of the Sick” the first four | paragraphs of the rubric minister, | the fifth and last ordains that “after confession | the priest shall absolve him after this sort, a J. O.
 
 
speak ot the
| Ihave a copy of the Book of Common Prayer, | . ‘Pp rin te ~d by Thom Buc k and R wer Daniel, | printers to the Universitie of Cambridge,” 4to., | 1637, and the Absolution in the Morning Prayer is to be pronounced by the “ Minister.” In the Communion Service the rubric before the General Confession runs as follows: “Then shall this general confession be made in th ume r: all | those that are minded to receive the Holy Com- munion, either b y one of them, or el by one of
the minist ers, or by the priest himself, all kneel- ig,” &e. Do the words “ either by one of them” occur in any other edition ?
? This edition of 1637 I believe,
 
    
lI
scarce, not bein ng in the Lea Wilson and tion.
A. BATEMAN. Randolph Gardens, W.
The rubric to the Absolution in the Book of
Common Prayer of 1637 says :—
“The absolution or remission of sinnes to be pro- nounced by the Presbyter alone, he standing up, and turning himself to the people, but they still remaining humbly upon their knees.”
M.. ¥.
In answer to Mr. Dore, the only Prayer Book I have between 1636 and 16:
“London: Printed by Robert Barker, Printer to the King’s most excellent Majesty: and by the Assignes of John Bill. 1638.” 12mo.
In this, the term rubric prefixed to the
39 is—
 
“Priest” is used in the
Absolution. _

---

sim_notes-and-queries_1876-07-29_6_135 - 1876-07-29, vol 6 iss 135 p.90-1

90
NOTES AND QUERIES.
 
(5S. VI. Juxx 29, ‘76,
 
on by inordinate ambition, he resolves on devoting a further term of years to the study of the in- tricacies of jurisprudence, and to abstruse and dogmatic questions of religion. Upon the com- pletion of the Moolazimat (residence as Moolazim) he is passed, after a serious and solemn examina- tion,] to the degree of Muderris (professor), which is the second degree of the Uléma, and also con- ferred by the Sheikh-ul-Islam in person. Once in possession of the professorial diploma the two magisterial careers are open to him, and he can demand the appointment of Mufti in a Mevleiiet (or a jurisdiction over twenty-four (cazas) courts of justice), the acceptance of which office, however, precludes all right to any ulterior advancement.
If by preference he remain inscribed on the list of professors, he must pass through the ten pro- fessorial classes, from that which is termed “outer” to that which is called “ Suleymanié.”** He is now raised to the third or highest degree of the Uléma, assumes the distinctive appellation of Molla Makhredji,tt and is duly qualified to be appointed either Molla of Islimbél, Kazi-Asker of Roomelia, or even Sheikh-ul-Islam, the elevation to which distinguished posts only awaits the good- will (ourf) and pleasure of the Sultan.
These three degrees of the Turkish University form so many connecting links of the important chain of the Uléma, as may be seen from the sub- joined table :—
Sserku-vt-IsuaM.
Tnte rpreters of
Ulima.
Judges—Fi ve Orders. Ministers y
the Law. Religion. Highest Degree. First Order, comprising six classes—1. Kazi-Asker of ** Mollas.”’ Roomelia; 2. Kazi-Asker of Anatolia; 3. Mollas of Islambol; 4. Mollas of Mecca; 5. Mollas of the Four Cities (viz., Adrianop'e, Broussa, Beyrout, Cairo) ; 6. Mollas Makhredji (¢.¢. having the right of preferment—and these are the Mollas of Galata, Eyoob, Smyrna, Yenicheher, Salonica, and Aleppo). Intermediate Degree. Second Order—Mollas Devriié (a). Sheikhs. “ Muderris.” Muftis. Third Order—Moofettish (2). Lowest Degree. Fourth Order—Kadis (or Kazis). Khatibs (c).
“ Moolaizim.”
Fifth Order—Naibs.
(a) These Mollas belong to the second rank of the magistracy. Each in turn fulfils certain duties, according to his rank, without presuming upon preferment; hence the name Devriié, “going round,” “ moving in &
circle.”
(b) The Moofettish are special judges, who decide upon all disputes relating to the Vacouf (pious bequests). (c) The inferior orders of the Ministers of Religion, the Imaums, Muezzins, and Kayims, form no part what-
ever of the Uléma.
From this tabular synopsis it is easy to perceive that a hierarchical and judicial body so powerfully organized, embracing the principal vital forces of Islam, and forming an aristocracy of position and not of birth, would, from its nature, be opposed to all constitutional reform.
To carry out everywhere the tanzimat of Gulk- hané {ft and subsequent ordonnances bearing on reorganization would be the ruin, the annihilation of their order. The pious bequests (Evkaf) and endowments by the Sultans at different epochs, which comprise two-thirds of the meneetal pro- perty in Turkey, and bring profit to the mosques without enriching the State, would be placed under
 
 
§ Examination, Imtihan; Examiner, Moomeyyiz.
** The ten degrees of Muderris are :—1. Kharidjih ; 2. Hareketh Kharidjih; 3. Dakhil; 4. Hareketh- Dakhil ; 5. Moussilé-y-Sahhn ; 6 Sabhn ; 7. Altmischly ; 8. Ikindjy-Altmischly; 9. Moussilé-y-Suleymaniyé ; 10. Suleymanié. Exch degree requires a fresh diploma (Rowous) indicating the precise aaree of the individual.
tt Makhredji, “ Un mot qui indique l'extraction des candidats de l’ordre de Muderriss, et leur agrégation dan: celui de Molla.”—D’Ohsson, tom. ii. p. 256.
tt Gulkhané, 3rd Nov., a.p. 1839, a.u. 1255, more fully expressed by tanzimat Khairiié, “felicitous organization.”
the general law of taxation, and apportioned off for the benefit of the empire at large.
The Uléma know their position full well, and, like every power based on illegitimate principles, cling to their authority, and endeafour to maintain it by the same means which contributed to estab- lish and consolidate it, illustrating the politician’s axiom :—“ Nemo malis artibus quesitum imperium bonis exercuit.” Wituiam Part.
Conservative Club.
A Fotx-Lore Soctery (5" §. v. 124, 294, 457 ; vi. 12, 37.) —“ Every dog has his day,” and as this (July 15) is St. Swithin’s, I think I am justified in regarding it as my most favourable opportunity for saying a little more about the proposed Folk- Lore Society.
The countenance An Otp Forx-Lorist gives to the scheme is very encouraging ; I am grateful for it, though he and I differ essentially as to the object the Society should have in view. This, with him, should be the printing of such items of folk-lore as have not been “already recorded by Brand and his editors, Sir Henry Ellis and Mr. Halliwell -Phillipps”; with me, the gathering

---

sim_notes-and-queries_1876-07-29_6_135 - 1876-07-29, vol 6 iss 135 p.90-1

st
od - of of
‘5 is in ty k-
to ‘or he
1s
by [r.
ng
 
f° 8. VI. Jury 29, '76.]
NOTES AND QUERIES. 91
 
 
together of all folk-lore, whether English or foreign, whether known to Brand or, if I may say so, brand-new to literature, the classifying of the materials, and the publication of the result in instalments, with as much learned note and com- ment as may be obtainable.
In this way only, I venture to think, could the Society materially help on the study of comparative folk-lore, or work with full benefit in other respects to those interested in its existence. I do not see why we should cast a slight on Dr. Brown, F.S.A., and the Rev. —— Jones (both of whom have pub- lished works on folk-lore) by accounting certain items of their carefully chronicled collections as “unrecorded,” merely because Brand and his editors lacked information of which Messrs. Brown and Jones have become possessed. The English Dialect Society carries on its investigations on the strength of half-guineas ; may not the Folk- Lore
Society hope to do the same, and thereby secure | | with Irish folk-lore.
many supporters, who might hold aloof in these “hard times” if membership involved the payment of a higher sum ?
I quite agree with An O_p Fotk-Lorist “ that
not a day should be lost in organizing” the Society; |
and I believe he and many others will agree with me that some important position therein should be offered to the original Folk-lorist of the Atheneum, the ex-editor of the “learned, chatty, useful ” periodical in which the present scheme was mooted, Mr. Wituiiam J. Tuoms. Is it too much to ask him to take some active preliminary steps in the cause—to play the grandfather to the bantling of his own offspring, “N. & Q.”?) Sr. Swirary.
I concur with several correspondents of “N. & Q.” that no time should be allowed to be lost in establishing this society. A special department might be constituted for the collection and recording of Celtic folk-lore, the superstitious beliefs and practices and the other popular lore of Ireland, Wales, the Scottish Highlands, and Man, —items which are interesting from their distinctive character, and the more urgently in need of pre- servation because associated in most cases with perishing vernacular tongues. In connexion with the useful list given by Mr. Rarciirre, and con- tinued by others, it may be mentioned that several Highland publications devote space to the subject of folk-lore, e.g. the Inverness Courier, the High- lander, and the Celtic Magazine (Inverness, A. & W. Mackenzie) ; and that the Gaelic Society of Inverness have appointed a folk-lore committee. Some interesting contributions to the subject were contained in the Gaidheal, a magazine published by Messrs. Nicolson, of Glasgow. It is now, I fear, extinct, and as I have received no reply to a communication addressed to the publishers, I pre- sume certain back numbers which I require cannot be supplied by them. Should the work be in the
| hands of any reader of “ N. & Q.,” he would confer a favour by lending the numbers in question (June, 1873 ; June, 1874; August, 1874; all previous to November, 1872, and all subsequent to Sep- tember, 1874), or any of them, for inspection, or, if preferred, I should be glad to purchase them.
Welsh folk-lore finds a home in the columns of the Oswestry Advertiser (from which it is reprinted as “ Bye-gones relating to Wales and the Border Counties”), in the Cambrian News, and, I believe, in other newspapers of the Principality. It is sig- nificant, as an illustration of the position occupied at present by Ireland as regards matters literary, that that country—so singularly rich in monuments of the past of every description, whether in the ruins and ancient remains meeting the traveller on all hands or in the beliefs and usages which have come down from a period beyond history —has not, so far as Iam aware, a single newspaper or other weekly or daily publication which concerns itself Apart from any other consi- deration, it seems strange that the managers of Irish newspapers do not recognize the increased interest which they would lend to their respective journals, and the possibility of increased circula- tion, by opening a folk-lore column.
Davip FirzGERA.p.
Hammersmith.
May I say how gladly I shall co-operate in the formation of a Folk-Lore Society, and that I will give any help in my power towards the speedy realization of such a project ?
James Britten, F.L.S.
British Museum.
Hamyet Suaxkspeare (5S, v. 461.)—Hamnet was formerly a very common name in Lancashire, judging from instances of its appearance in my note-books. I have not facilities for giving gene- ral details, but the following particulars may be of interest to your able correspondent, with whose paper before me I cannot venture to alter the orthography in the documents cited. On the Stretford register, extending from 1598 to 1711, Hamnet occurs as follows—(a) as a baptismal name: Sept. 27, 1607, “ Johan’ filius Hamnetz Harwood” ; Nov. 5, 1609, “ Bricheta Herwood filia Hamletti Herwood”; Nov. 12, 1648,“ James Hunt, the sonne of Hamblet Hunt” (the latter as Hamnett Hunt appears upon a list now before me, consist- ing of the inhabitants of Stretford in the year 1641-2); June 13, 1652, “ Penalabee hunt the dauter of hamlit hunt” (this child, “ Penelope hunt the dauter hamnit Hunt,” was buried Sept. 26 following) ; Dec. 10, 1655, “ Mary the dauter of hamlit Hunt” (buried, under the same spelling, ten years afterwards, Aug. 20); Sept. 8, 1672, “ Ham Lit the sonne of James Hunt,”—(d) as a sur- name : Dec. 1, 1667, “ Ann the davter of Edward
 
Hamlit”; Feb. 7, 1674-5, “ Edward the son of

---

sim_notes-and-queries_1876-08-12_6_137 - 1876-08-12, vol 6 iss 137 p.137-8

5* 8. VI. Ava. 12, '76.] NOTES AND QUERIES. 137
 
third, sixth, and ninth vespers, and the compline or post-communio . that is, of seven different hours, on account of that saying of David, Septies in die laudem divi tibi.” The London Encyelo- dia (1829) defines it in very nearly the same nguage. It is curious that the blunder should not be in the third edit. of Hook’s Church Dic- tionary, whilst it is in the fifth edit. It is not in the current edition of the work. mm FP.
pe la
PROVINCIALISMS FoR “To Torasn” (5 §. vy. 426: vi. 56.\—Mr. WiILkrnson omits a common word in the north of England, skelp.
E. Leaton BLenxkrxsorr.
Lapy ARABELLA Dewxy (5 S. y. 346, 456. Since Lord Edmond Fitzmaurice’s interesting Lift of Lord Shelburne shows that this great and good lady, his ancestress, was, although a very gentle- tempered woman, a sufficiently strong-minded one to check with a loaded pistol (the only argument of any use ¢ t such an animal) the brutish
iwains
insults of a foolish and vicious man, it is plain that this story should be forgotten except in thos peculiar social circles, outside or half outside ones,
for and by whose pure and refined tastes it was | J I
invented. Lady Arabella Denny was the pa troness and organizer of many charitable institu tions besides the one her name is oftenest connected with. She improved the old workhouse schoo and system generaily in Cork and Dublin. Her insight into character and her business faculties were recognized by the corporations of both cities who presented her with the freedom of their boroughs), and were as remarkable as her practical
benevolence. Hipernia. * Creetne” (5" §, vi. 48, 96.)—The following cutting from a glossary of the dialects of the
wapentakes of Manley and Corringham, co. Lincoln, which I have now in the press, answers Mippte Tempar’s question :—
‘ Cree, v. to simmer grain until tender. ‘Squire alus gives his hosses creed lineseed, that’s why they their coats.’
“ Creed-wheat, wheat simmered until tender, and eaten with sugar and spices, or made into ‘ Frumenty.’’
Epwarp Peracock.
 
sottesford Manor, Brigg.
Weatuer Hotes (5™ §. v. 88, 176, 435.)
here is a place in Shropshire which may be classed with the other recorded instances of weather holes. People in the neighbourhood of Shrewsbury say, “ There ’Il be some rain, for the wind has got into Habbe rley Hole.” Habberley is a village about seven miles south-west of Shrews- bury, a point from which rain generally comes, so that the character which this place has ac quired is due to its accidental geographical position. Why the word “ hole” should be used in connexion with the saying I could never understand. Perhaps
some of your correspondents might be able to explain. W. H.
Shrewsbury.
Poems spy Mrs. Parmer (5 S. v. 495; vi. 54.) —I was in error in stating that Mrs. Palmer wrote poems in the Devonshire dialect. The volume I referred to was similar to the one in Mr. Pen- GELLY’S possession, i.c. A Dialoque in the Devon- shire Dualect, in prose. Three editions of this little book have been published—viz., in 1837, 1839, and as a Devonshire Courtship in 1869—in each of which there is a glossary. W. N.
 
“ Regvies curarum” (5% §. v. 385, 523.)—Let }me quote the following beautiful passage from | Tibullus in illustration :
 
| “* Sic ego secretis possim bene vivere sylvis, Qua nulla humano sit via trita ped
| Tu mihi curarum re juies, tu nocte vel atra, Lumen, et in solis, tu mihi turba locis.”
It is thus paraphrased by Tom Moore :—
Charm of my life, by whose sweet power All cares are hushed, all ills subdued,
| My light in e’en the darkest hour,
My crowd in deepest solitude.”
Joun Picxrorp, M.A.
| Newbourne Rectory, Woodbridge.
 
7, 495 ; vi.
] | 58.)—I remember hearing the late Dr. Neale say,
| Prorane Hymn Teves (5 §. v. ¢
ind I think it will be found also in some of his | writings, that the Scotch Reformers, finding the | people attached to their old Church music, and | persisting in singing it, were afraid that this prac- | tice would hinder the work of Protestantizing the nation. They therefore hit upon the plan of pro- ing profane songs to be sung to the old Church melodies. In some instances the words actually ridiculed the sacraments and services of the Church. Many of the favourite old airs are derived from Church hymn tunes.
E. Leaton BLENKINSOPP.
  
The tune ** Belmont” is not from Mozart, but is Haydn’s “ My mother bids me bind my hair.” It was adapted by 8. Webbe, some time organist of the then Sardinian Embassy Chapel in London, | to the hymn, “ As pants the hart,” ec.
T. R. Grunpy.
Newton A!
A Forx-Lore Society (5S. v. 124, 294, 457; vi. 12, 37, 91.)—The bibliography of the subject would form an important division of the projected society’s publications ; and, having made extensive collections respecting the superstitions and reli- gious beliefs of mankind generally, I should be happy to afford assistance in that field of its labours.
A half-guinea subscription would tend to give | numerical importance to the society, and enlist in

---

sim_notes-and-queries_1876-08-12_6_137 - 1876-08-12, vol 6 iss 137 p.137-8

138
NOTES AND QUERIES.
 
[5" 8. VI. Ava. 12, 76.
 
its service many contributors who might otherwise stand aloof; while the subscribing for several copies would enable those to promote its financial success who are able and willing.
The plan of aggregating the folk-lore of races or countries in separate books would scarcely, I fear, answer, since it would either indefinitely delay the society’s issues, or tend to split them, as additional materials accumulated, into a multitude of insigni- ficant portions. It would appear more advisable to classify according to subject (“ Days and Sea- sons,” “Spells and Charms,” &c.), and to issue in fasciculi, with a continuous pagination, notes under each heading, duly labelled, in Mr. Skeat’s phrase, “after the manner of the mineralogist, with the place of collection,” leaving further classification to very full synoptical indexes. This method would enable the society to commence its pub- lications at once, and thereby foster and increase the ardour of its members, which long delays would as certainly quench. Tuo. SaTcnet.
Oak Village, N.W.
Mr. Firzcerap (p. 91) mentions the Oswestry
Adverlizr, and also the Cambrian News, as containing each week a column of folk-lore, under the head of “Bye-Gones.” I have had
the pleasure of editing this column from the com- mencement, in 1871, and I write this to suggest to the conductors of other papers, who publish antiquarian columns, that they reprint the matter in a convenient form for binding. This may be done very cheaply by using the type already set up for the newspaper, and the number may be regulated by the price charged. We find a hun- dred copies (which are subscribed for by contri- butors and local libraries) to be sufficient for the purpose we have in view, i.e. the preservation of the matter in good hands for future use. Askew Roperrs. Croeswylan, Oswestry.
“Tae GouLpEN VaniTEE” (5% §. vi. 69, 99.) —The song inquired for by R. H. is somewhat long, but as much of its bulk typographically is caused by repetitions and refrain, it may be com- pressed into reasonable space. Many years ago I heard it sung with spirit and grave mirthfulness by Mr. Fraser, of the Scottish Society of Antiquaries, whose property it virtually became by right of conquest, in Edinburgh. It is not improbably an old convivial song, but I never saw it in print earlier than the date of Mrs. Gordon’s memoir of Christopher North, published by Messrs. Edmon- ston & Douglas, in 1862. She received the words and music of the song (which had delighted Pro- fessor Wilson) from her friend P. S. Fraser. The “ Eek iddle dee” was doubtless a whimsical ex- = of the sharp scrape of the fiddle in the
ands of some supposititious old Crowdero. It was repeated after each second line ; and the first
 
line was given with variations. As may be seen, there are manifold differences between the single verse given by R. H. and the Edinburgh complete copy — “* There was a gallant ship, Anda Ss ship was she, k iddle dee, and the Lowlands low ; And ae was called ‘the Goulden Vanitee As she sailed to the Lowlands low. She had not sailed a league, A league but only three, Eek, &c. When she came up with a French Gallee, As she sailed, X&c. Out spake the little Cabin-boy, Out spake he, Eek, Ke. ; > ‘What will you give me if I sink that French Gallee?’ Out spake the Captain, Out spake he, ‘We'll gi’e ye an estate in the North Countree.’ ‘Then row me up ticht, In a black bull's skin, And throw me o'er deck-buird, sink I or swim.’ So they ‘ve row'd him up ¢icht, In a black bull's skin ; And have thrown him o'er deck-buird, sink he or swim. About and about, And about went he, Until he came up with the French Gallee. Oh ! some were playing cards, And some were playing dice ; When he took out an Instrument, bored thirty holes at twice ! Then some they ran with cloaks, with caps, To try if they could stap the saut-water draps. ¢ About and about, And about went he, Until he cam back to the Goulden Vanitee. * Now throw me o'er a rope, And pu’ me on buird ; And prove unto me as guid as your word.’ ‘We'll no throw you o’er a rope, Nor pu’ you on buird, Nor prove unto you as guid as our word.’ Out spoke the little Cabin-boy, Out spoke he, ‘Then hang me /°Ul sink ye as I sunk the Gallee.’ But they ve thrown him o'er a rope, And have pu'd him up on buird ; And have proved unto him far better than their word: As they sailed by the Lowlands low.” Whosoever may have been the author of the ditty, he knew well the ingratitude of a nation to its benefactors and defenders, and certainly antici- pated the power and efficacy of our Naval Steam Rams. If they are only as destructive to the enemy as they seem inclined to be to our own seamen, there will be something gained for the outlay :-— “ Por it showed what a very fine Ram she was, Said the Ad-mi-ral-i-tee.’ The author of “The Goulden Vanitee” remains unknown (Memoir of John Wilson, ii. 317) ; but, it is to be hoped, further information m: ay be gained. J. W. E. Molash, by Ashford, Kent.
Cow Fotk-Lore (5™ S. v. 349; vi. 109.)—An instance of the oak or supposed, sapusienity of
And some they ran
French
”

---

sim_notes-and-queries_1876-09-02_6_140 - 1876-09-02, vol 6 iss 140 p.198

198
NOTES AND QUERIES.
 
[5" 8. VI. Serr. 2, ’76,
 
One Year in Sweden (1862) mentions the Skene of Rubislaw who was enrolled amongst the Swedish noblesse, his nobility as an armiger having been acknowledged. The appendix to that work con- tains a list (more or less correct) of “Scotch Nobles” (of Sweden), titled and untitled ; in the former class, I presume, Mr. Skene included him- self, as I suppose the expression “ take his seat among the Swedish nobility” is equivalent to ours of “a seat in the Lords.” In Sweden, as in Ger- many, titles, I believe, descend to ali the heirs male ; therefore are not the Lords of the Swedish Parliament the heads of the titled families ? C. S. K.
Kensington.
There is appended to Horace Marryat’s One Year in Sweden a long list of Scotsmen who have been enrolled amongst the nobility of Sweden.
Ap ColILuvs.
St. Stephen's Club.
“ Creeine ” (5 §, vi. 48, 96, 137.) —The French phrase for to boil rice, or other grains, soft is, “Faire crever du riz.” Crever literally means to burst out. Could the provincial verb to cree be kindred to the French word ? Henri GAUSSERON.
Ayr Academy.
“Sxip” (5™ §. iv. 129, 335, 371 ; v. 117, 337; vi. 97, 119.)—I am agreeably surprised to find that Pror. Fiske hasnot allowed this discussion todrop. In 5" 8. v. 117, he launched his shafts at my * ignorance,” now (vi. 97) he denounces my * sup plementing or testing” authority as “ utterly un- trustworthy ” and “ full of errors,” without ascer- taining who compiled that work, and takes care to claim his own as being “one of the best.” Allow me to quote from another work published in Stockholm by the Printer to the University, written by a Swede in English, p. 207, “ Another mode of communication to which most of the poor people resort is skidfart, a sort of skating.” How about the well-known roller-skate? Would the Professor term that skid or skridsko ?
Liverpool.
ProvincraLisms ror “To Turasn” (5 §, vy. 426 ; vi. 56, 137.)—A few years back I was pass- ing down a street in Dumfries, just as a school was “ skailing,” and heard the following dialogue, which may serve to illustrate Mr. BLENKrINsopp’s note. One small boy, who had not been to school, said chaffingly to another who was coming out “ greetin’,” “Eh! Sandy lad, an’ hae ye bin skelpit the day ?”—“ Ay, weel that.”
7 A oe
The words bannick, jacket, and fight may be added to the list already given. They will be
 
found in the Rev. W. D. Parish’s Dictionary of | the same place.
the Sussex Dialect, and are still current in this of Surrey. G. L. Titsey Place, Redhill.
Rev. W. Braxton (5 S. v. 107, 216, 521; vi. 57, 118.)—Thanks to the kind communications of your correspondents, I am enabled to give further notes on a portion of the Blakiston pedigree in continuation of Surtees’s pedigree, vol. iii. p. 164, and in explanation of the notice of the recent marriage of Miss Florence Blakiston Dunn, as mentioned in my last note (5" 8. vi. 118).
Robert Blakiston, of Bishop Wearmouth, co, Durham, living in 1821, when Surtees compiled his pedigree, was a descendant from Marmaduke Blakiston, the founder of the Newton Hall (Durham) and Old Malton (Yorkshire) branches of the Blakistons. Surtees places this Marma- duke as fifth son of John Blakiston, Esq., of Blakiston (co. Durham), and not third, as alleged by Mr. Jonnson Batty (5" S. v. 216). This Marmaduke was one of a family of sixteen (ten and six daughters). Robert Blakiston married Margaret Cass at Bishop Wearmouth, Nov. 1, 1785, living in 1821. They had issue— William, bap. 1789; James, bap. 1790 ; Robert, bap. 1793 ; Mary, bap. 1787 ; Eleanor, bap. 1792 (died unmarried in 1810) [perhaps some corre- spondents can supply me with the names of the descendants of the first four children, with dates, &c.]. Margaret Tempest Blakiston, another of Robert Blakiston’s children, bap. May 26, 1795, married the Rev. James Dunn, B.D., Rector of Preston, Suffolk, and had issue John Dunn, Esq., of Kirby Lodge, whose eldest daughter, Florence Blakiston Dunn, was married on June 29, 1876, at Little Shelford, Cambridge, to William Donkin, Esq., Oxford. Anne Tempest Blakiston, another daughter of Robert Blakiston, was born June 18, 1796 (qy. if ever married?). Harriet Tempest Blakiston, another daughter of Robert Blakiston, was born Feb. 28, 1799 ; married, first, Alexander Renney, merchant, of Riga, in Russia, by whom she had a daughter, Juliana Margaret Renney (married to Dr. Anderson, C.B., of Sunnybrae, Pitlochry, Perthshire), and a son, G. A. Renney, Lieut.-Col. R. Horse Artillery. Harriet Tempest Blakiston’s second husband was Thomas Gray, Esq., of Sunderland, by whom she had a large family. 8. F. Lonestarre, F.R.H.S.
Norton, Stockton-on-Tees.
part G.
sons
A Foxrx-Lore Soctery (5 §. v. 124, 294, 457; vi. 12, 37, 90, 137.)}—To make such a society a success its working committee must not consist ex- clusively of Londoners, or such as can attend meetings in the metropolis. And I would suggest that, as far as its annual meeting is concerned, the society (if formed) shall be a wandering one, that is to say, it shall not meet twice consecutively in H. Fisuwick, F.S.A.

---

sim_notes-and-queries_1877-01-27_7_161 - 1877-01-27, vol 7 iss 161 p.77

mS, VII. Jax. 27,

---

sim_notes-and-queries_1877-05-12_7_176 - 1877-05-12, vol 7 iss 176 p.375

] yi p” A Yaranthor | Ge for f } ‘ +} dD minister at + forth
 
 
 
_ VIL. Mar 12,77.) NOTES
pes
AND QUERIES.
   
 
ArcupisHop Rotnernam’s Buriat-Piace (5" §. vii. 292, The following extract is from Brown’s excellent History of York:
“ The acc
ing dies d« f the
yunt by Mr. Drake (of the archbishop hav- ue), which is not supported by any seem to be consistent with fact. that the archbishop died of the
     
pl authority, does
There is no evide
 
nee
             
    
 
   
 
 
 
xe, or that this dreadful visitation extended its res beyond the year 1499. No memorial remains of nediate or hurried funeral, or of the burning of ecording to the tale with which the exhi- iti f the head is accompanied in the cathedral . said to have been sumptuously and honour ably interred ; but it is certain that the solemn exequies of the Church woul en performed over an empty coffin During the nth of Ja wary, A r of the eastern f the centra r was removed, and removal cau > ma le in the west f the l ] 8 circun n bled ] ir ins} e va and thus to obser Ther 1 was dis play le. ce of violent destruction and violation, fora st coffin was evidently much broken, and reof lay by the sides of the lead coffin 1 open its whole length lleft ina ] rag lisordered stat ¥ | bones of the bis i tot ty perfect ribs not having se} i remains wel 1g with the feet to the east ‘The t al the vault contai r tl remains of are hop suffered very much by the destructive f 1829, but in 1832 it was restored, except in some f the marks of its anci it adornments, by tl liberality f the Rect vl Fellows of Lincoln Colle ford, whom t archbishop was a munificent benefactor.’ G. Rotherha BETWEEN you anp I” (5" §. vii. 138, 254.)— Tn letter of Charl K ley, dated Helston, May 16, Charles Kingsley, his Letters and Me / Life, vol p 29), this expression ( 8 f ** Between you and I Mr. C.’s botany must be at a very low ebb”; “But Mr C. wrong-names his plants dr idfully between you
s then only sixteen. G.
In tl rst act of Goldsmith’s She Stoops to ‘Mi ’ ‘Wor 1 give papa and | a little of your company ?”
oe Ws.
es Lvcas (5 S. vii. 67, 99 T)
. Hardeastle says to
nere
 
n that Lord de Grey did privately _ it a memoir of Sir Charles Lucas, inasmuch as Lay y The resa Lewis. in her LL 3 of ti Friends a iC 7 wporaries of Lord C] neellor Cl rendon vol. ii, p 85), quotes from it a letter, dated Col- chester, Aug. 27, 1648, and ads sed to Lord x by Sir C. Lucas and others. If the in- quirer is at ted in the life of Sir C. Lucas or
ry of hester, he may advantageously hee 8 + ime of the work of Lady Theresa ». 67-165, 240-79
 
W. Pripeavx CourtTNey.
Queen Anne’s Gate, Westminster.
 
A Forx-Lore Society (5 S. vy. 124, 294, 457
vi. 12, 37, 90, 137, 198; vii. 77.)—May I ask if any action is being taken in this matter? There can be little doubt, I fancy, but that, were it gene- rally known such a society was in process of formation, many would be anxious to join it. There is such obvious need for it, that there can be surely only one opinion as to its suc- cess. I sincerely trust the hope of ig the Folk-Lore Society has not been al
 
Wii
‘Toe Martyr 01 we ; vu. 57, 116.
ErromManca” (5% §. vi. 368
A Narrative of Missionary Enterprises in the South Sea Islands. By hn Williams, of the London Mis- flonary Society Eighth T isand Published for the Author by J. Snow, 26, Paternost Row, London. 1292” Such is the title of my 8vo. ve ‘ As the work was extremely popular, and became still more so ifter Williams's death, I have no doubt it is most ly obtained. Fr. ke “ Bouentren” (5t® S, vi. 488 ; v 1] This
word is in provincial New England, and those portions of the ed States settled by New Englanders,* in ense mentioned. Let
 
me here remark that l au- thority for New England provincialisms, but not for words “ generally used in America.”
ro- AMERICUS,
 
‘Cram” (5S. vi. 246, 296, vil. 59) in olnshire means to take h nd is
There is a tule told of a Methodist
  
mp
 
in ordinary use. . local light ’ preaching on the wrestling of Jacob, who, getting thoroug rhly warmed to his subject, ex- claimed * Ves, my brother ters he ? ? ee = } } ” clammed owd o’ th rel Ww nt ie im goa, Clams are also a large pair of double iron hooks, hung loosely at the end of a r pe « chain, and which fall together and so fasten themselves into bales, &c., while being wound or drawn up
tubs,
'
by cranes or pulleys. Boste n, Lincolnshire.
R. R.
 
BANKS AND HIS Horse Morocco (5 §. vi. 387, 176.) ‘“ Old Banks the juggl er, one Pythag .
Grave tutor to the learned horse; both which
Jeing beyond sea, burned for one witch,
Their spirits transmigrated to a cat; And now, above the pool, a face rig With great grey eyes, it lifted up, ar Thrice did it spit, thrice dived.
, * *
t fat, d mewed
They cried out ‘Puss!’ He told them he was Banks, That had so often showed them merry pranks.” Ben J neon, On the Fas s Voyage.
en led to the top of Gull’s Horn-
Morocco is reported to have as¢ Paul’s, London.
St. Dekker says,
: . i Americanism
glander an
* Is, or ig not, the term Zn

---

sim_notes-and-queries_1877-10-13_8_198 - 1877-10-13, vol 8 iss 198 p.298

NOTES AND QUERIES.
 
Riding of Yorkshire both as a verb and a noun; remple, ?
but ha ys a very different signification from pre either of those given by your correspondents | in vy R. R. and Mr. BiewKry It means in that | clos to tl down kly or fall « | wor enly Thu It 3 with He \ ] itl wn in a tantrur k ly H. E. W S , Anerley S.E tl .
l ©} t reverence I ( iI L} His ¢ lral A ‘ ht} } . “th t C | } | " LD) 5 ¢, 214.) | I eM f ’ / M l } I 1H. B E. E. T. S.., « | Le | ‘ : : ( An | 4 t whyle I le in tl } at | 7 saw t fend tt la long ind a greats . y t vs r l vy t t , r, and vy l bysely pel l l l ¢ 1 fayl ; } f wy -} ¥ | t + } LI lvu 1
|
Being vol. viii. of the Manx Society's pub
Vi INIA (5S. viii. 27, 76, 152 The cele- brated divine, William Crashaw, preacher of the
i
‘ec Lor l De law ire, 1
 
indsey means a cock of any breed, 0
Lucirer Marcues As these useful little articles have noticed
 
     
5th S, V
 
mention I recolle 1848. 1 length o! bluish st was pecu of action rdinary two or | pape BR | and held in the ri through It was p sometim
“Lea TO THE 938, 2 A post li memoral the wril
S. Wilb
58
made a with GE
> eis ht.
ment embe nk which I | I sh GENE Ox
m FF. yulber

---

sim_notes-and-queries_1877-12-01_8_205 - 1877-12-01, vol 8 iss 205 p.421

gS. VIII. Dec. 1, '77.]
NOTES AND QUERIES.
421
 
LONDON, SATURDAY, DECEMBER 1, 1877.
CONTENTS.—N?® 205.
NOTES :—A Folk-Lore Society, 421—Shakspeariana, 422— Rob: Robe: Rubbish, 423—Original Letter, 425—Col Francis Charteris’s Will: Popular Superstition at his Death —Inquisitions Post Mortem—An Epitaph in Youlgrave Charchyard—Ducking-Stools, 426.
QUERIES :—Robert Earl of Leicester's Daughter—Sir Julius Cesar (Master of the Rolls)—Crest of Waring Family—De Canci Family—A Deficiency of Language—Cousins, 427— Pope Calixtus I — Marguerite, Duchess of Bretagne, &c.— “Catalogue of 500 Celebrated Authors”"—‘* News”—A Stoneing Cross—Antlers of the Red Deer—A ‘‘Snow”—The Ulster Riband, 428—Windsor Castle—Milton Papers—Sir W. Withers —Dr. Williams, the Oculist—Peter Steven Schull —Hunt, the Translator of Tasso’s “Jerusalem,” 429— Authors Wanted, &c., 430.
REPLIES :—Bonvyle Family, 430—The Sunflower, 431— Mahomet in Flight, 432—‘‘ A monkey on the house "—The “Bride Stones,” Yorkshire—A Mysterious Phrase, 433—The Montague Peerage—‘‘ Fool, fool,” &c.—The Rev. C. Herle —Blessing the Fishing, 434—Works on Trading Routes— “The Bothie of Tober-na-Vuolich”—** Witword"’: ‘‘ Caup- land "—The Halsham Family, 435—Isolda : Gladys—Devon Provincialisms—‘‘Chic"—Old Roman Inscription, 436— Place Names—Sheridan's Begum Speech—The (reat Frosts of 1783, &c, 437—Massinger and Alfred de Musset—A Fodder of Lead—Chess—“‘ Ye Gowden Vanitee "—The Us« of the Cope—Nagares, 438S—Authors Wanted, 439.
Notes on Books, &c.
fotes. A FOLK-LORE SOCIETY.
Those only who know the fascination which the subject of popular antiquities, superstitions, and traditions has had for me from the time I read Sir Francis Palgrave’s learned and amusing articles upon it in the Quarterly Review, and Sir Walter Seott’s charming essay on fairy mythology, in his Minstrelsy of the Scottish Border, more than fifty years ago, can understand the pleasure with which I read, in “N. & Q.” of Feb. 12, 1876, Sr. Switnin’s happy suggestion of a folk-lore society. From causes with which it would be an imperti- nence to trouble your readers, I was unable to do more than wish the proposal God-speed, which I did in “N. & Q.” of July 1 following, in a short paper signed An Orv Forx-Lorist, which con- tained sufficient indications as to who the writer was ; at all events I believe, from the manner in which Sr. Swirnty (vi. 91) paid me the compli- ment of suggesting I should play grandfather to the bantling of my own offspring, “ N. & Q.,” that there was at least one who recognized me.
Nothing but the reasons to which I have just |
referred prevented my responding to so flattering aM invitation, and placing the experience in the management of such a society which I had acquired during my thirty-four years’ connexion with the first of all such societies, the Camden, at the ser- Vice of the folk-lore loving public.
 
But non sum qualis eram. My readiness to work in so good a cause is not backed by an equal ability ; and I fear a correspondence I have had with Sr. Swirnry, hampered as it was by the conflict between my will and my ability, has done more harm than good by putting off its formation ; and it is by way of apology for this unintentional hindrance, and to promote the good work, that I venture to trouble you with my ideas as to how that good work may be most effectually accomplished.
There are two points on which my opinion is unchanged :—
1. That the headquarters of the society must be in London.
2. That the subscription must be 11. a year.
I insist on the first point because the Folk-Lore Society must not only have a local habitation and a name, but that habitation must be central and permanent. For, unlike the vrinting societies, the books of which are circuleted and dispersed, the few remainders being stored at the printers’, the Folk-Lore Society will be a society to collect and store a vast amount of curious, out-of-the-way, old-wives’ lore. One of its most important duties will be to gather in the various local journals which indulge in folk-lore columns, and to mount and preserve those columns for future use. I will not now stop to suggest the very simple mode in which I think this may be accomplished.
Do not let it be supposed, from my insisting on London as a centre, that I undervalue the import- ance of local co-operation. How much the success of the Camden Society was owing to the intro duction of local secretaries (which, if I remember rightly, arose from a suggestion of Dr. Bliss, who became local secretary for Oxford) a glance at the first annual list of members would show ; while the Society of Antiquaries, which some years afterwards followed the example of the Camden Society, has now upwards of seventy gentlemen, many of the highest rank, who act as local secre taries to that learned body.
I insist upon the subscription being 17. because I foresee that, from the nature of the society, the incidental expenses, apart from the paper and printing, must be large ; and paper and print ar luxuries which must be paid for.
Next to the gathering together the relics of our own folk-lore, and eventually the analogous folk lore of other countries, the printing and circulation of the more important items will be the chief object of the society.
There would, on the one hand, be great economy in confining this to the publication of an annua! volume.
On the other, though there are obvious objec tions besides that of increased expense, there would probably be some compensating advantages by smaller and more frequent issues, something like the Zeitschrift fiir deutsche Mythologieund Sittenkunde,

---

sim_notes-and-queries_1878-01-12_9_211 - 1878-01-12, vol 9 iss 211 p.39

5% 8. LX.
y be for
   
also * Going, g
are both t
As to the «
stand to ea
l S.) for 185
be found in R S " E
Jan. 12, 78.)
NOTES AND QUERIES.
39
 
ind .
in Grahar Maga Barbour’s Bruce,
welcome to tl instalment, | will he English Works of J we congratulate tl Society that ted t an ed labours have shown justice to the life prelate who seule | cerely hope that
ve (Philadelphia, —
by u
> subs
th S. viii. 5 veing
 
Alack aday ! the well is dry,” &e.
ne ! to Tom Toddl
 
for seven pounds ten,’ W.
DP a book R.
 
ent
 
ix. 1
(5'» S. viii. 599; *« Pity is akin to love.”
Qj
 
xact relationship in which love and pity
other, there seems to be
 
 
. some doubt | J Mr Dp } work here commence Pity, some say, is the parent , ' ’ Of future love Mr. J. ¢ und Fletcher, The Spanish C Act v, | portant and , Devbush 1e say pity is the child of love.” This third v Cotton, 2 T y.5 Appletree S Swit! only well wr — . printed, and ght is ler 1 Dryden or Bu 8 | bound ll Y rember S 3 7 j AN daaaih wemeeue is I ty - i me ries in sor Ol. 17 6 a ceg! ) love visit thes ( D. B. I revert have published a secor L a Lett f Hiscellanecous. ccmie hes We & “rae = NOTES ON BOOKS, &e. Ma wulay's 8 = in 8 est t each
[ he I
 
 
the
members of the | of t
ind 1 his
 
Mr. Skeat—and as such wi ribers. The fourth is
first part of a collection
1 be most ikxewise an
which
» special interest for many readers, namely, 7h«
F and Text
eculiar characte s been or, Mr. Mayor, whose }
how especially fitted writings otf
 
ly English
  
 
   
 
rue | is faith 1 Mayor w 1 sooner ites vol. ii f or ndon, B iITrose the Hundreds of rhe wor s not 1 rauiy ly ves of Wil revive nd exe in others iret n ents, and V yously f their Messrs. L id issue of the s edition Vaca It is empha t vise ¢t S¢ V ) i note n books, and also how 1 vere, the very st k of any er

---

